In [ ]:
from spswarehouse.warehouse import Warehouse
from spswarehouse.table_utils import *
from bs4 import BeautifulSoup
from ducttape.webui_datasource import WebUIDataSource
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from future.utils import raise_with_traceback
import pandas as pd
import time
from datetime import date, datetime
import os
from tempfile import mkdtemp
import shutil
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementNotVisibleException,
)

from ducttape.utils import (
    interpret_report_url,
    wait_for_any_file_in_folder,
    get_most_recent_file_in_dir,
    DriverBuilder,
    LoggingMixin,
)
from ducttape.exceptions import (
    InvalidLoginCredentials,
    ReportNotFound,
    NoDataError,
)

REPORT_GENERATION_WAIT = 10

#Grab today's date, as well as the current timestamp
now = datetime.now()
time_stamp = now.strftime("%Y_%m_%d__%H_%M_%S")
TODAY = now.strftime("%m/%d/%Y")


class SummitLearning(WebUIDataSource, LoggingMixin):
    def __init__(self, username, password, wait_time, hostname='summitlearning.org', temp_folder_path=None,
                 headless=False, login_provider='google'):
        super().__init__(username, password, wait_time, hostname, temp_folder_path, headless)

        self.hostname = hostname
        self.login_provider=login_provider
        self.uri_scheme = 'https://'
        self.base_url = self.uri_scheme + 'www.' + self.hostname

    #This _login function was borrowed from the 'ducttape' repo
    def _login(self):
        if self.login_provider == 'google':
            login_url = self.base_url + '/auth/google_oauth2'
            self.driver.get(login_url)

            #Google has more than one type of login screen; this try-except block
            #tries to use the 'Email' one first
            try:
                elem = self.driver.find_element_by_id('Email')
            except NoSuchElementException:
                elem = self.driver.find_element_by_id('identifierId')
            elem.clear()
            elem.send_keys(self.username)
            elem.send_keys(Keys.RETURN)
         
            elem = WebDriverWait(self.driver, self.wait_time).until(
                EC.visibility_of_element_located((By.ID, 'username'))
            )

            elem.send_keys(self.username)
            elem.send_keys(Keys.RETURN)
            
            elem = WebDriverWait(self.driver, self.wait_time).until(
                EC.visibility_of_element_located((By.ID, 'password'))
            )
            
            
            elem.send_keys(self.password)
            elem.send_keys(Keys.RETURN)


        elem =  WebDriverWait(self.driver, self.wait_time).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'VfPpkd-RLmnJb')))

        elem.click()
            
        # wait for the destination page to fully load

        WebDriverWait(self.driver, self.wait_time).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'app-teacher')))
            

    def download_url_report(self, stu_df, write_to_disk=None, **kwargs):
    
        #These are the column names that are in the new output table
        column_names = ['first_name', 'last_name', 'scraped_date', 'checkin_date',
                        'checkin_notes', 'student_id', 'mentor_group_id', 'email',
                        'platform_student_id', 'academic_year', 'reported_site_short_name',
                        'teacher_id', 'teacher_email', 'teacher_first_name', 'teacher_last_name']
        
        #Create a new dataframe with the above columns
        output_df = pd.DataFrame(columns=column_names)
        
        curTeacher = ''

        if write_to_disk:
            csv_download_folder_path = write_to_disk
        else:
            csv_download_folder_path = mkdtemp()

        #Create instance of Selenium driver in order to navigate through SLP website
        self.driver = DriverBuilder().get_driver(csv_download_folder_path, self.headless)
        
        #Call ._login() in order to sign into the SLP using the service account
        self._login()
        stuCount = 0
        
        
        #Iterate through the table pulled from the Warehouse, in order to
        #visit every student's page in the SLP, and pull any notes from their
        #checkins
        #Note: The table has been sorted alphabetically by mentor group email,
        #so that the code does not have to constantly switch between "viewing as"
        #different mentors.
        for index, row in stu_df.iterrows():
            
            
            stuCount += 1
            curFirstName = row['first_name']
            curLastName = row['last_name']

            curStudentId = row['student_id']
            curMentorId = row['mentor_group_id']
            curEmail = row['email']
            curPlatId = row['platform_student_id']
            curAcademicYear = row['academic_year']
            curSite = row['reported_site_short_name']
            curTeacherId = row['teacher_id']
            curTeacherFirst = row['teacher_first_name']
            curTeacherLast = row['teacher_last_name']
            
            checkTeacher = row['teacher_email']
            
            #Uncomment this if you'd like to have a visual check that the code is
            #making progress
            #print(checkTeacher, stuCount)
           
            firstCheck = False
            
            #This code deals with changing which mentor you are "viewing as";
            #The code runs a series of clicks in order to do this.
            if checkTeacher != curTeacher:
                if curTeacher == '':
                    firstCheck = True
                else:
                    firstCheck = False
                    
                curTeacher = checkTeacher
                self.driver.get("https://www.summitlearning.org/")

                if not firstCheck:
                    self.driver.get("https://www.summitlearning.org/auth/as/noone?source=stop")

                elem = WebDriverWait(self.driver, self.wait_time).until(
                    EC.visibility_of_element_located((By.CLASS_NAME, 'avatar'))
                    )
                elem.click()

                elem = WebDriverWait(self.driver, self.wait_time).until(
                        EC.visibility_of_element_located((By.ID, 'usernav'))
                    )
                
                elem.click()    

                elem = self.driver.find_element_by_link_text('View as...')
                elem.click()        

                elem = WebDriverWait(self.driver, self.wait_time).until(
                    EC.visibility_of_element_located((By.CLASS_NAME, 'rbt-input-main'))
                    )
                elem.click()

                elem.send_keys(curTeacher)

                elem = WebDriverWait(self.driver, self.wait_time).until(
                    EC.visibility_of_element_located((By.ID, 'rbt-menu-item-0'))
                )

                elem.click()

                WebDriverWait(self.driver, self.wait_time).until(
                    EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Viewing as')]")))            
    
                
            #This puts together the current student's URL, that contains checkin notes
            urlString = "https://www.summitlearning.org/teacher/students/" + str(curPlatId) + "/mentee_snapshot"
            self.driver.get(urlString)
            
        
            theHtml = self.driver.page_source
            
            
            soup = BeautifulSoup(theHtml, 'html.parser')
            stuName = soup.find('h1', {'class':'app-title'}, text=True)
            
            #Checks to see if the URL works, if it doesn't, the code goes on
            #to the next student
            try:
                stuName = stuName.text
                theText = soup.findAll('div', {'class':'snapshot-checkin-panel-info'})
                
                
                #'theText' should contain a list of all of the student's checkins
                #This for loop iterates through all the checkins, and if the checkin
                #is not empty, then a new row is added to the output table, containing
                #the relevant information.
                for checkin in theText:
                    curText = checkin.findAll(text=True)

                    if len(curText) > 2:
                        noteText = curText[-1]
                        if noteText == " There are no notes from this check-in." :
                            noteText = ""

                        splitNoteDate = curText[0].split(" ")
                        noteDate = splitNoteDate[0]
                        
                        newEntry = {
                            'first_name' : curFirstName,
                            'last_name' : curLastName,
                            'scraped_date' : TODAY,
                            'checkin_date' : noteDate,
                            'checkin_notes' : noteText,
                            'student_id' : curStudentId,
                            'mentor_group_id' : curMentorId,
                            'email' : curEmail,
                            'platform_student_id' : curPlatId, 
                            'academic_year' : curAcademicYear, 
                            'reported_site_short_name' : curSite,
                            'teacher_id' : curTeacherId,
                            'teacher_email' : checkTeacher,
                            'teacher_first_name' : curTeacherFirst,
                            'teacher_last_name' : curTeacherLast
                        }
                        output_df = output_df.append(newEntry, ignore_index=True)
                        
            except Exception as e:
                pass
        
        #Close the Selenium driver
        self.driver.close()
        
        return output_df            

In [ ]:

#Get password from 'cred.txt' file
with open("cred.txt", "r") as f:
    pw = f.readline()
f.close()


#This is the query that will be executed on the Warehouse database, to grab an initial pool of students
sps_query = '''

query removed for security purposes

'''

#Here, we are creating an instance of the SummitLearning class,
#using the team's service account account to log into the SLP
slp = SummitLearning('account_censored', pw, 600)

#This is the output of the Warehouse query, stared as a pandas dataframe
sps_students = Warehouse.read_sql(sps_query)

#This calls the function that runs the scraper
#slp_report = slp.scrape_checkin_notes(sps_students)
slp_report = slp.download_url_report(sps_students)

#Puts together log filename
output_file_name = "checkin_log_" + time_stamp + ".csv"

cur_pwd = os.getcwd()
log_dir_path = os.path.join(cur_pwd, "logs")

#Check to see if 'logs' folder exist; if it doesn't
#the code creates one
if not os.path.exists(log_dir_path):
    os.mkdir(log_dir_path)

#Puts together full pathname for log file
final_log = log_path + output_file_name
final_log_path = os.path.join(log_dir_path, output_file_name)

#Writes output table to log file
slp_report.to_csv(final_log)


In [ ]:
#Visual check of output table
slp_report